In [14]:
import re
    
text = open('ling_emails.txt','r', encoding = 'utf-8').read()
messages = re.findall('ЭТОМОЁПИСЬМОНОМЕР:[0-9]*(.+?)ЭТОКОНЕЦПИСЬМАНОМЕР', text, flags = re.DOTALL)

pronouns = open('pronouns.txt','r', encoding = 'utf-8').read()
slovar = open('zdf-win.txt','r', encoding = 'utf-8').read()
abbrf = open('abbreviations.txt', 'r', encoding = 'utf-8').read()
abbr = abbrf.split('\n')
l = len(abbr)
for x in range(l):
    abbr.append(abbr[x].lower())
print(abbr)

['Ул.', 'Д.', 'Корп.', 'Каб.', 'М.', 'Пер.', 'Руб.', 'e.g.', 'т.к.', 'гг.', 'г.', 'тыс.', 'тысч.', 'англ. ', 'д.э.н.', 'к.ф.н', 'д.ф.н', 'ауд.', 'нояб.', 'дек.', 'янв.', 'апр.', 'сент.', 'окт.', 'фев.', 'п.', 'к.', 'т.е.', 'л.', 'к.и.н.', 'см.', 'прил.', 'стр.', 'в.н.с.', 'мин.', 'ауд.', 'г.', 'д.', 'М.', 'ул.', 'см.', 'пер.', 'Б.', 'каб.', 'д.э.н.', 'т.е.', 'тел.', 'корп.', 'т.к.', 'им.', 'т.', 'стр.', 'к.и.н.', 'В.', 'к.', 'м.', 'т.п.', 'п.', 'янв.', 'См.', 'гг.', 'Г.', 'руб.', 'студ.', 'тыс.', 'дек.', 'П.', 'Т.', 'О.С.', 'Ауд.', 'Стр.', 'Л.', 'к.ф.н.', 'К.', 'филос.', 'Р.Я.', 'мин.', 'н.', 'Т.е.', 'рег.', 'Ч.', 'англ.', 'мат.', 'ст.', 'доб.', 'М.б.', 'УЛ.', 'наст.', 'Т.к.', 'л.', 'окт.', 'а.', 'С.Р.', 'ж.', 'худ.', 'сент.', 'го.', 'в.н.', 'прил.', 'Франц.', 'к.э.', 'орг.', 'гр.', 'муз.', 'иностр.', 'им.А.', 'экзамен.', 'У.', 'зам.', 'соц.', '﻿ул.', 'д.', 'корп.', 'каб.', 'м.', 'пер.', 'руб.', 'e.g.', 'т.к.', 'гг.', 'г.', 'тыс.', 'тысч.', 'англ. ', 'д.э.н.', 'к.ф.н', 'д.ф.н', 'ауд.',

In [3]:
def cleaner(text):
    text1 = ''
    for l in range(len(text)): #убираем разметку жирного шрифта
        if text[l] != '*'or (text[l] == '*' and text[l-1] in '0123456789' and text[l+1] in '0123456789'):
                text1 += text[l]
        
    lines0 = text1.split('\n')
    n = 0
    lines = []
    for i in lines0:
        if i != '':
            lines.append(i)
    i = 0
    j = 0
    while i != len(lines):
        if len(re.findall('[0-9]*:[0-9]* пользователь', lines[i])) > 0 or len(re.findall('On Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday, [0-9]+ September|October|November|December|January|February|March|April|May|June|July|August [0-9]+, ', lines[i])) > 0 or len(re.findall('_____', lines[i])) > 0 or len(re.findall('Отправлено со', lines[i])) > 0 or len(re.findall('Sent from', lines[i]))>0 or len(re.findall('> ', lines[i])) > 0 or len(re.findall('---- Пересылаемое сообщение ----', lines[i]))>0 or len(re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2} [0-9]{2}:[0-9]{2} GMT', lines[i]))>0 or len(re.findall('[0-9]{2}\.[0-9]{2}\.[0-9]{4}, [0-9]{2}:[0-9]{2}, "', lines[i]))>0 or len(re.findall('пользователь.*написал', lines[i]))>0:
            lines = lines[:i]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        if len(re.findall('Копия:', lines[i])) > 0:
            lines = lines[i+1:]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        i += 1
    lines_str = ''
    for j in lines:
        lines_str += j
        lines_str += ' '
    return lines_str

In [4]:
def rev(x):
    y = ''
    for i in reversed(x):
        y += i
    return y

In [5]:
def sklonenie(x):
    d = re.search('[0-9]+([а-яА-Я]+)', x)
    if d is not None and d.group(1) in ['го', 'им', 'ым', 'му', 'ом', 'ем', 'ый', 'ий', 'й', 'я', 'е']:
        return True
    else:
        return False

In [6]:
def lstrip(x):
    punct = []
    while len(x) > 0 and x[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',', '«', '»', "№", '[', ']', '\u200b', '“', '=']:
        if x[0] == '.':
            punct.append(re.match('\.+', x).group(0)) #нашли все многоточия (в начале и в конце)
            x = x.lstrip('\.+') #убрали многоточие
        elif x[0] == '?' or x[0] == '!':
            punct.append(re.match('((!|\?)+)', x).group(0))
            x = x.lstrip('((!|\?)+)')
        elif x[0] == ')':
            punct.append(re.match('\)+-*:*', x).group(0)) #проверка на веселые смайлики
            x = x.lstrip(re.match('\)+-*:*', x).group(0))
        elif x[0] == '(':
            punct.append(re.match('\(+-*:*', x).group(0)) #проверка на грустные смайлики
            x = x.lstrip(re.match('\(+-*:*', x).group(0))
        elif x[0] == ':' and x[1] == '(':
            punct.append(re.match(':-*\(+', x).group(0))
            x = x.lstrip(re.match(':-*\(+', x).group(0))
        elif x[0] == ':' and x[1] == ')':
            punct.append(re.match(':-*\)+', x).group(0)) #проверка на веселые смайлики
            x = x.lstrip(re.match(':-*\)+', x).group(0))
        elif x[0] == '\u200b':
            x = x[1:]
        else:
            if x not in [']', ';', ':', '"', '-', '—', '*', ',', '«', '»','[', '$', '%', '“', '=']:
                p = re.match(']|\)|;|:|"|-|—|\*|,|\(|«|»|№|\[|“|=', x).group(0)
                punct.append(p)
                x = x.lstrip(p)
            else:
                return x, list(reversed(punct))
    num = re.search('([0-9]+[а-яА-Яa-zA-Z]+\.{0,1})', x)
    if sklonenie(x) != True and num is not None and num.group(0) == x:
        num1 = re.search('([0-9]+)([а-яА-Яa-zA-Z]+\.{0,1})', x)
        x = num1.group(2)
        punct.append(num1.group(1))
    return x, punct

In [7]:
def rstrip(x):
    punct = []
    x1 = rev(x)
    while len(x1) > 0 and x1[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',', '«', '»', '[', ']', '%', '$', '”', '=']:
        if x1[0] == '.':
            if init(rev(x1)) != True and red(rev(x1)) != True and cifra(x1[1:]) != True:
                punct.append(re.match('\.+', x1).group(0)) #нашли все многоточия (в начале и в конце)
                x1 = x1.lstrip(re.match('\.+', x1).group(0)) #убрали многоточие
            else:
                break
        elif x1[0] == '?' or x1[0] == '!':
            punct.append(re.match('((!|\?)+)', x1).group(0))
            x1 = x1.lstrip(re.match('((!|\?)+)', x1).group(0))
            x = rev(x1)
        elif x1[0] == ')':
            punct.append(re.match('\)+-*:*', x1).group(0)) #проверка на веселые смайлики
            x1 = x1.lstrip(re.match('\)+-*:*', x1).group(0))
        elif x1[0] == '(':
            punct.append(re.match('\(+-*:*', x1).group(0)) #проверка на грустные смайлики
            x1 = x1.lstrip(re.match('\(+-*:*', x1).group(0))
        
        else:
            if x1 not in [']', ';', ':', '"', '-', '—', '*', ',', '«', '»','[', '$', '%', '”', '=']:
                p = re.match(']|;|:|"|-|—|\*|,|«|»|\[|$|%|”|=', x1).group(0)
                punct.append(p)
                x1 = x1.lstrip(p)
                x = rev(x1)
            else:
                return x1, list(reversed(punct))
    num = re.search('([0-9]+\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
    if num is not None and num.group(0) == x1:
        num1 = re.search('([0-9]+)(\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
        x1 = num1.group(2)
        punct.append(num1.group(1))
    for i in range(len(punct)): #перевернуть скобки обратно
        punct[i] = rev(punct[i])
    return rev(x1), list(reversed(punct))

In [8]:
def red(x):
    reduces = ['д.', 'доб.', 'каб.', 'ул.', 'тел.', 'м.', 'пер.', 'руб.', 'e.g.', 'т.к.', 'гг.', 'г.', 'тыс.', 'тысч.', 'англ.', 'д.э.н.', 'к.ф.н.', 'д.ф.н.', 'Inc.', 'ауд.', 'ноябр.', 'дек.', 'янв.', 'апр.', 'сент.', 'окт.', 'фев.', 'п.', 'к.', 'т.е.', 'л.', 'к.и.н.', 'см.', 'прил.', 'стр.', 'в.н.с.', 'мин.', 'М.', 'т.п.', 'P.S.', 'м.б.', 'им.', 'к.', 'н.', 'филос.', 'напр.', 'пр.']
    red_occ = re.search('([А-Я]\.)', x)
    if x in reduces or x.lower() in reduces:
        return True
    elif red_occ is not None:
        if red_occ.group(0) == x:
            return True
    else:
        return False

In [9]:
def cifra(x):
    if x in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X']:
        return True
    else:
        return False

In [10]:
def abbrev(words):
    if x in abbr:
        return True
    elif x.strip('.') in abbr:
        return True
    else:
        return False

In [12]:
def init(x):
    flag = False
    s = re.search('([А-Я]\.[А-Я]\.)', x)
    if s is not None:
        if s.group(0) == x:
            flag = True
    return flag

In [13]:
def interval(x):
    s = re.search('([0-9]+-[0-9]+)', x)
    if s is not None and s.group(0) == x:
        return True
    else:
        return False

In [15]:
def house(x):
    s = re.search('[0-9]+/[0-9]+', x)
    if s is not None and s.group(0) == x:
        return True
    else:
        return False

In [16]:
def streets(words):
    streets = open('double_street_name.txt','r').read()
    streets1 = streets.split('\n')
    a = []
    for i in streets1:
        a.append(i.split(' '))
    words1 = []
    i = 0
    bi = 'abracadabra'
    while i < len(words)-1:
        word = words[i]
        flag = 1
        bi = words[i] + ' ' + words[i+1]
        for j in a:
            if len(words[i]) > 3 and len(words[i+1]) > 3:
                m1 = re.match(words[i][:-3], j[0])
                m2 = re.match(words[i+1][:-3], j[1])
                if (len(words[i+1]) > 3) and (m1 is not None) and (m2 is not None):
                    word = bi
                    flag = 2
                    break
        words1.append(word)
        i += flag
    words1.append(words[-1])
    return words1

In [17]:
def phone(words):
    words1 = []
    i = 0
    while i < len(words)-2:
        word = words[i]
        flag = 1
        tri = words[i] + ' ' + words[i+1] + ' ' + words[i+2]
        if i < len(words)-3:
            quat = words[i] + words[i+1] + words[i+2] + ' ' + words[i+3]
            third = re.match('[0-9]+', words[i+1])
            forth = re.match('[0-9]{3}-', words[i+3])
        first = re.match('\+{0,1}[0-9]+', words[i])
        second = re.match('[0-9]{3}-', words[i+2])
        if i < len(words)-3 and words[i] == '('and third is not None and third.group(0) == words[i+1] and words[i+2] == ')' and forth is not None:
            word = quat
            flag = 4
        elif first is not None and first.group(0) == words[i] and words[i+1] == '-' and second is not None:
            word = tri
            flag = 3
        words1.append(word)
        i += flag
    if i < len(words)-1:
        words1 += words[i:]
    return words1

In [18]:
def add(words):
    words1 = []
    i = 0
    while i < len(words)-3:
        word = words[i]
        flag = 1
        quat = words[i] + ' ' + words[i+1] + ' ' + words[i+2] + ' ' + words[i+3]
        first = re.search('[0-9]+', words[i+1])
        second = re.search('[0-9]+', words[i+3])
        if words[i] == 'доб.' and first.group(0) == words[i+1] and words[i+2] == '-' and second.group(0) == words[i+3]:
            words1.append('доб.')
            word = words[i+1] + ' ' + words[i+2] + ' ' + words[i+3]
            flag = 4
        words1.append(word)
        i += flag
    if i < len(words)-1:
        words1 += words[i:]
    return words1

In [19]:
punctuation = ['.', ',', '?', '!', ')', '(', ']', '[', '-', '_', '#', '%', ':', ';', '"']

In [21]:
def tag_checker(word):
    tag = ''
    if word == '' or word == None:
        tag = 'empty'
    elif len(word) == 1 and word in punctuation:
        tag = 'punctuation'
    else:
        cyr = re.search('[а-я]|[А-Я]', word)
        lat = re.search('[a-z]|[A-Z]', word)
        up = re.search('([A-Z]|[А-Я])', word)
        low = re.search('([a-z]|[а-я])', word)
        num = re.search('[0-9]', word)
        if cyr and lat and num:
            tag = 'super-lex-numb'
        elif cyr != None and lat == None and num == None:
            tag = 'cyr'
            initial = re.search('([А-Я]{1})\.([А-Я]{1})\.', word)
            if initial:
                tag += ' initial'
        elif cyr == None and lat != None and num == None:
            tag = 'lat'
        elif cyr != None and lat != None and num == None:
            tag = 'mixed'
        elif cyr != None and lat == None and num != None:
            tag = 'cyr-num'
        elif cyr == None and lat == None and num != None:
            tag = 'numbers'
            time =  re.search('[0-9]{2}\:[0-9]{2}', word)
            if time:
                tag = ' time'
            date = re.search('([0-9]{1,2}(\:|\.|\-| )[0-9]{1,2}(\:|\.|\-| )[0-9]{2})|([0-9]{1,2}(\:|\.|\-| )[0-9]{1,2}(\:|\.|\-| )[0-9]{4})', word)
            if date:
                tag = ' date'
            phone = re.search('(8|\+7)(-|-\(|\()([0-9]{3,10})(-|-\(|\()', word)
            if phone:
                tag = ' phone'
            nazv = re.search('«.{,15}?»', word)
            if nazv:
                tag = 'name of smth'
        if word in abbr:
            tag += ' abbreviation'
        if up and low:
            lp = 0
            if word[0] == word[0].upper():
                for w in range(1, len(word)):
                    if w == word[0].upper():
                        lp += 1
                if lp == 0:
                    tag += ' lex-prop'
        if word[0] == '#':
            tag += ' hashtag '
        elif word[0] == '@':
            tag += ' @-tag'
        link = re.search('(http(s)\:|www\.)', word)
        if link:
            tag = 'link '
        mail = re.search('.+?@[a-z]{3,7}\.[a-z]{2,3}', word)
        if mail:
            tag = 'mail'
    if tag == '':
        tag = 'something'
    return tag

In [22]:
class Token:
    def __init__(self, name = '', tag = '', t_id = 0, l_id = 0):
        self.name = name
        self.tag = tag
        self.l_id = l_id
        self.t_id = t_id
    def output(self):
        return str('\nToken ' +  str(self.t_id) +': \'' + self.name + '\' \nType: ' + self.tag + '\nDocument: letter ' + str(self.l_id) + '\n')

In [23]:
def splitter(text):
    if text == '':
        return ''
    
    text = cleaner(text)
    words = text.split() #по умолчанию делим по пробелу, единые сущности, разделенные пробелом, будут рассмотрены потом
    words2 = []
    
    for x in range(len(words)):
        lstr = lstrip(words[x]) #стрип слева
        if len(lstr[1]) != 0:
            words2 += lstr[1] #знаки препинания перед словом
        rstr = rstrip(lstr[0]) #стрип справа
        if '(' in rstr[0]: #проверка на скобку внутри слова, часто перед ней нет пробела
            words2.append(rstr[0].split('(')[0])
            words2.append('(')
            words2.append(rstr[0].split('(')[1])
        elif '/' in rstr[0]  and 'www' not in rstr[0] and 'http' not in rstr[0] and 'ru' not in rstr[0] and house(rstr[0]) != True: #проверка на слеш внутри слова
            words2.append(rstr[0].split(re.search('/+', rstr[0]).group(0))[0])
            words2.append(re.search('/+', rstr[0]).group(0))
            words2.append(rstr[0].split(re.search('/+', rstr[0]).group(0))[1])
        elif ',' in rstr[0]: #проверка на запятую внутри слова
            words2.append(rstr[0].split(',')[0])
            words2.append(',')
            words2.append(rstr[0].split(',')[1])
        elif '-' in rstr[0]:
            if interval(rstr[0]) == True or (rstr[0] not in pronouns and rstr[0] not in slovar):
                words2.append(rstr[0].split('-')[0])
                words2.append('-')
                words2.append(rstr[0].split('-')[1])
            else:
                words2.append(rstr[0])
        else:
            words2.append(rstr[0]) #итоговое слово
        if len(rstr[1]) != 0:
            words2 += rstr[1] #знаки препинания после слова 
    words3 = []
    for i in words2:
        if i != '':
            words3.append(i)
    
    words3 = streets(words3) #соединение единых токенов, разделенных пробелом (улицы, телефоны)         
    words3 = phone(words3)
    words3 = add(words3)
    if len(words3) > 0:
        if words3[-1] == '-':
            words3 = words3[:-1]
    return words3

In [40]:
def abbr_freq(tokens): #задание на частотный список сокращений
    freq_abbr = []
    for x in testo:
        ab = re.search('(([а-я]|[А-Я]){1,}\.)+', x.name)
        if ab:
            freq_abbr.append(ab.group(0))
    freq_a = Counter(freq_abbr)
    return freq_a #далее результат выдачи функции был очищен вручную

In [28]:
def main(letters):
    t_id = 0
    tokens = []
    for l in range(5, 40):
        words = splitter(letters[l])
        for word in words:
            t_id += 1
            tag = tag_checker(word)
            token = Token(word, tag, t_id, l)
            tokens.append(token)
    return tokens

testo = main(messages)
for t in testo:
    print(t.output())


Token 1: 'Уважаемые' 
Type: cyr lex-prop
Document: letter 5


Token 2: 'студенты' 
Type: cyr
Document: letter 5


Token 3: '!' 
Type: punctuation
Document: letter 5


Token 4: 'Если' 
Type: cyr lex-prop
Document: letter 5


Token 5: 'Вы' 
Type: cyr lex-prop
Document: letter 5


Token 6: 'не' 
Type: cyr
Document: letter 5


Token 7: 'можете' 
Type: cyr
Document: letter 5


Token 8: 'дозвониться' 
Type: cyr
Document: letter 5


Token 9: 'в' 
Type: cyr
Document: letter 5


Token 10: 'бухгалтерию' 
Type: cyr
Document: letter 5


Token 11: ',' 
Type: punctuation
Document: letter 5


Token 12: 'чтобы' 
Type: cyr
Document: letter 5


Token 13: 'заказать' 
Type: cyr
Document: letter 5


Token 14: 'деньги' 
Type: cyr
Document: letter 5


Token 15: 'с' 
Type: cyr
Document: letter 5


Token 16: 'депонента' 
Type: cyr
Document: letter 5


Token 17: ',' 
Type: punctuation
Document: letter 5


Token 18: 'просьба' 
Type: cyr
Document: letter 5


Token 19: 'звонить' 
Type: cyr
Document: letter 5


To

Частотный список сокращений

In [47]:
{'ауд.': 105, 
 'г.': 99, 
 'д.': 67, 
 'М.': 58, 
 'ул.': 47, 
 'см.': 46, 
 'пер.': 37, 
 'Б.': 28, 
 'каб.': 25, 
 'д.э.н.': 24, 
 'т.е.': 22, 
 'тел.': 22, 
 'корп.': 21, 
 'т.к.': 18, 
 'им.': 17, 
 'т.': 17,
 'стр.': 12, 
 'к.и.н.': 12, 
 'В.': 11, 
 'М.И.': 11, 
 'к.': 11, 
 'М.А.': 11, 
 'С.': 10, 
 'м.': 10, 
 'т.п.': 10, 
 'п.': 10, 
 'янв.': 10, 
 'См.': 9, 
 'Р.': 9, 
 'Е.': 8, 
 'О.И.': 8, 
 'гг.': 8, 
 'Г.': 7, 
 'руб.': 7, 
 'Н.Е.': 7, 
 'студ.': 6, 
 'Э.': 6, 
 'А.М.': 5, 
 'В.А.': 5, 
 'Ю.А.': 5, 
 'тыс.': 5, 
 'Е.В.': 5, 
 'Н.А.': 5, 
 'Д.А.': 5, 
 'дек.': 4, 
 'П.': 4, 
 'Т.': 4, 
 'О.С.': 4, 
 'Ауд.': 4, 
 'А.С.': 4, 
 'М.В.': 4, 
 'Стр.': 4, 
 'Л.': 4, 
 'А.А.': 4, 
 'И.М.': 3, 
 'И.': 3, 
 'Ю.': 3, 
 'Д.': 3, 
 'О.': 3, 
 'В.М.': 3, 
 'к.ф.н.': 3, 
 'К.': 3, 
 'В.В.': 3, 
 'С.А.': 3, 
 'А.Д.': 3, 
 'филос.': 2, 
 'Р.Я.': 2,
 'мин.': 2, 
 'н.': 2,
 'Т.е.': 2,
 'рег.': 2, 
 'Ч.': 2, 
 'англ.': 2, 
 'мат.': 2,
 'ст.': 2,
 'доб.': 2, 
 'М.б.': 2,
 'УЛ.': 1,
 'наст.': 1, 
 'Т.к.': 1, 
 'л.': 1, 
 'окт.': 1, 
 'а.': 1,
 'С.Р.': 1,
 'ж.': 1, 
 'худ.': 1, 
 'сент.': 1,
 'го.': 1,
 'в.н.': 1,
 'прил.': 1,
 'Франц.': 1,
 'к.э.': 1,
 'орг.': 1,
 'гр.': 1,
 'муз.': 1,
 'иностр.': 1,
 'им.А.': 1,
 'экзамен.': 1, 
 'У.': 1,
 'зам.': 1,
 'соц.': 1}
 

{'А.А.': 4,
 'А.Д.': 3,
 'А.М.': 5,
 'А.С.': 4,
 'Ауд.': 4,
 'Б.': 28,
 'В.': 11,
 'В.А.': 5,
 'В.В.': 3,
 'В.М.': 3,
 'Г.': 7,
 'Д.': 3,
 'Д.А.': 5,
 'Е.': 8,
 'Е.В.': 5,
 'И.': 3,
 'И.М.': 3,
 'К.': 3,
 'Л.': 4,
 'М.': 58,
 'М.А.': 11,
 'М.В.': 4,
 'М.И.': 11,
 'М.б.': 2,
 'Н.А.': 5,
 'Н.Е.': 7,
 'О.': 3,
 'О.И.': 8,
 'О.С.': 4,
 'П.': 4,
 'Р.': 9,
 'Р.Я.': 2,
 'С.': 10,
 'С.А.': 3,
 'С.Р.': 1,
 'См.': 9,
 'Стр.': 4,
 'Т.': 4,
 'Т.е.': 2,
 'Т.к.': 1,
 'У.': 1,
 'УЛ.': 1,
 'Франц.': 1,
 'Ч.': 2,
 'Э.': 6,
 'Ю.': 3,
 'Ю.А.': 5,
 'а.': 1,
 'англ.': 2,
 'ауд.': 105,
 'в.н.': 1,
 'г.': 99,
 'гг.': 8,
 'го.': 1,
 'гр.': 1,
 'д.': 67,
 'д.э.н.': 24,
 'дек.': 4,
 'доб.': 2,
 'ж.': 1,
 'зам.': 1,
 'им.': 17,
 'им.А.': 1,
 'иностр.': 1,
 'к.': 11,
 'к.и.н.': 12,
 'к.ф.н.': 3,
 'к.э.': 1,
 'каб.': 25,
 'корп.': 21,
 'л.': 1,
 'м.': 10,
 'мат.': 2,
 'мин.': 2,
 'муз.': 1,
 'н.': 2,
 'наст.': 1,
 'окт.': 1,
 'орг.': 1,
 'п.': 10,
 'пер.': 37,
 'прил.': 1,
 'рег.': 2,
 'руб.': 7,
 'сент.': 1,
 'см

Тестирование на корпусе >500 токенов

In [29]:
def main_test(letters):
    t_id = 0
    tokens = []
    for l in range(800, 806):
        words = splitter(letters[l])
        for word in words:
            t_id += 1
            tag = tag_checker(word)
            token = Token(word, tag, t_id, l)
            tokens.append(token)
    for l in range(810, 815):
        words = splitter(letters[l])
        for word in words:
            t_id += 1
            tag = tag_checker(word)
            token = Token(word, tag, t_id, l)
            tokens.append(token)
    return tokens

testo = main_test(messages)
for t in testo:
    print(t.output())


Token 1: 'Здравствуйте' 
Type: cyr lex-prop
Document: letter 800


Token 2: '!' 
Type: punctuation
Document: letter 800


Token 3: 'В' 
Type: cyr
Document: letter 800


Token 4: 'последний' 
Type: cyr
Document: letter 800


Token 5: 'момент' 
Type: cyr
Document: letter 800


Token 6: 'мы' 
Type: cyr
Document: letter 800


Token 7: 'обнаружили' 
Type: cyr
Document: letter 800


Token 8: 'несостыковку' 
Type: cyr
Document: letter 800


Token 9: 'с' 
Type: cyr
Document: letter 800


Token 10: 'расписанием' 
Type: cyr
Document: letter 800


Token 11: 'сессии' 
Type: cyr
Document: letter 800


Token 12: 'на' 
Type: cyr
Document: letter 800


Token 13: 'сайте' 
Type: cyr
Document: letter 800


Token 14: '(' 
Type: punctuation
Document: letter 800


Token 15: 'http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable' 
Type: link 
Document: letter 800


Token 16: '-' 
Type: punctuation
Document: letter 800


Token 17: 'courses=1&timetable' 
Type: someth

In [31]:
out = open('output.txt', 'w')
for t in testo:
    out.write(t.output())

Ошибки:
    1. часов).Во 
    2. а 
        )
    3. в
        10
        -
        3-
    4. tkulakova@hse.ru<mailto:tkulakova@hse.ru>

##### Точность: 0.99564
##### Полнота: 0.99132

Теперь деление на предложения

In [26]:
def splitter_sent(text):
    uppercase = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯABCDEFGHIJKLMNOPQRSTUVWXYZ'
    text = text.replace('\n', ' ')
    text1 = ''
    for i in range(len(text)):
        if text[i] != ' ' or i == 0 or (text[i] == ' ' and text[i-1] != ' '):
            text1 += text[i]
    text = text1
    sent = ''
    sents = []
    space = -1
    i = 0
    while i < len(text):
        if text[i] not in ['.', '!', '?']:
            if text[i] == ' ':
                space = i
            elif text[i] in uppercase:
                if text[i:i+5] in ['Дорог', 'Всем ', 'Здрав', 'Приве', 'Уважа']:
                    sents.append(sent.strip(' -'))
                    sent = ''
            sent += text[i]
            i += 1
        else:
            if i == len(text)-1 or (text[i+1] in [' '] and i+1 == len(text)-1) or (text[i+1] in [' '] and text[i+2] in uppercase) or (text[i+1] in [' '] and text[i+2] in ['-']):
                space = i+1
                sent += text[i]
                sents.append(sent.strip(' -'))
                sent = ''
                i += 2
            elif text[i+1] in uppercase:
                space_end = i
                while space_end < len(text):
                    if text[space_end] == ' ':
                        break
                    space_end += 1
                if not init(text[space+1:space_end]):
                    sent += text[i]
                    sents.append(sent.strip(' -'))
                    sent = ''
                    i += 1
                else:
                    sent += text[i]
                    i += 1
            else:
                sent += text[i]
                i += 1
    if sents == [] and len(sent) > 0:
        sents.append(sent)
    sents1 = []
    for i in sents:
        if i != '':
            sents1.append(i)
    return sents1

Тестирование деления на предложения

In [33]:
sents = []
for l in range(800, 806):
    sent = splitter_sent(messages[l])
    sents += sent
for l in range(810, 815):
    sent = splitter_sent(messages[l])
    sents += sent
print(sents)

['Здравствуйте!', 'В последний момент мы обнаружили несостыковку с расписанием сессии на сайте (http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399), где экзамен начинается в 9 , и тем, что было вывешено учебный офисом (10 часов).', 'Во сколько на самом деле нужно приходить?', 'Вся группа считала, что приходить нужно к 10.', 'Давайте в 10.', 'Только чтобы все пришли к 10 и чтобы не было путаницы.', 'Всего доброго, Борис Орехов 22 июня 2015 г., 0:50 пользователь lingua kingdom <linghse2014@gmail.com> написал: >', 'Здравствуйте!', 'В последний момент мы обнаружили несостыковку с > расписанием сессии на сайте > ( > http://www.hse.ru/ba/ling/timetable?fromdate=2015.06.22&todate=2015.06.27&groupoid=399&timetable-courses=1&timetable-groups=399 > ), > где экзамен начинается в 9 , и тем, что было вывешено учебный офисом > (10 часов).', 'Во сколько на самом деле нужно приходить?', 'Дорогие студенты, Завтра мы ждем вас В

In [37]:
out_sents = open('output_sents.txt', 'w')
for i in sents:
    out_sents.write(i)

##### Точность: 0.934
##### Полнота: 0.866